### Talk Time Data Prepare

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word usage
min_word_use_num = 1
max_word_use_num = 6

# youtube
sample_num = 1  # 6
time_shift = 0.6

In [4]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [5]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [7]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [8]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_threegram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [9]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/3-Talk Time Data Prepare"

Path(path).mkdir(parents=True, exist_ok=True)

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [13]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word.xlsx", index=False)

In [14]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [15]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
328541,zesentwintig horizontaal,3
328542,volledig gedisciplineerd,3
328543,volledig geconcentreerd,3
328544,hero netegengif,3


In [16]:
# Twogram Func
d_list2  = df_twogram_sent.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram_sent, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
6564,wel moeder,3
6565,te dood,3
6566,wel niks,3
6567,hier alleen,3


In [17]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_threegram_sent

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
550436,vliegen onbekend aantal,3
550437,vliegen obstakel afnemen,3
550438,voor achtduizend euro,3
550439,voor de kijkcijfers,3


In [18]:
# Threegram Func
d_list3  = df_threegram_sent.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram_sent, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
29304,maar goed anders,3
29305,waar over dan,3
29306,kom ben kom,3
29307,ja maar zeker,3


In [19]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:02.501,dit is de pangale v,BoaYsdPtJYA
1,00:00:04.501,00:00:05.434,unreal,BoaYsdPtJYA
2,00:00:08.518,00:00:13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,00:00:13.066,00:00:14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,00:00:14.316,00:00:15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,00:01:37.135,00:01:47.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,00:01:47.040,00:01:49.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,00:01:49.340,00:01:53.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,00:01:53.600,00:01:57.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [20]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [21]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA
1,4.501,5.434,unreal,BoaYsdPtJYA
2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [22]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx")
df_sentence_link = df_sentence_link.iloc[:,[0,1,2,3,4,5]]
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...
1,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
2,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
3,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...
4,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...
5,je wordt ook heel wat wel dan zou je allemaal ...,431,436,waarmee je wordt ook heel wat wel dan zou je a...,5bGpXBIbiXw,https://www.youtube.com/watch?v=5bGpXBIbiXw&t=...
6,als ze zeg maar op je af komen en of je weet g...,615,620,gewoon heel enge als ze zeg maar op je af kome...,Fn5erRPierY,https://www.youtube.com/watch?v=Fn5erRPierY&t=...
7,misschien nog even dank u zeggen aan de mensen...,226,230,misschien nog even dank u zeggen aan de mensen...,GMsJ12_DaK8,https://www.youtube.com/watch?v=GMsJ12_DaK8&t=...
8,vind voor die andere beter dan gaan we daar we...,101,106,deze toneel vind voor die andere beter dan gaa...,5pDwEvEN_dA,https://www.youtube.com/watch?v=5pDwEvEN_dA&t=...
9,een keer tegen mij zij die niet voor mij zijn ...,360,365,zei ooit een keer tegen mij zij die niet voor ...,LyPygt9GFlk,https://www.youtube.com/watch?v=LyPygt9GFlk&t=...


In [23]:
twogram_set = set()
threegram_set = set()
for sent in df_sentence_link["search_string"]:
    twogram_zip = ngrams(sent.split(), 2)
    for two in twogram_zip:
        twogram = " ".join(two)
        twogram_set.add(f"{twogram}") 
    threegram_zip = ngrams(sent.split(), 3)
    for three in threegram_zip:
        threegram = " ".join(three)
        threegram_set.add(f"{threegram}")

In [24]:
len(twogram_set)

260

In [25]:
len(threegram_set)

261

In [26]:
twogram_list = list(twogram_set)
#twogram_list

In [27]:
threegram_list = list(threegram_set)
#threegram_list

In [28]:
df_twogram_for_link = df_twogram_select[df_twogram_select["twogram"].isin(twogram_list)]
df_twogram_for_link.reset_index(drop=True, inplace=True)
df_twogram_for_link

,twogram,frequency
0,dank u,64014
1,nog niet,24775
2,ja ja,11987
3,niet nu,6282
4,doe maar,5449
...,...,...
128,echt heel,3
129,die we,3
130,ook wel,3
131,meer over,3


In [29]:
df_twogram_selected_for_link = word_usage_result(word_list, df_twogram_for_link, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_selected_for_link 

,twogram,frequency
0,dank u,64014
1,nog niet,24775
2,ja ja,11987
3,niet nu,6282
4,doe maar,5449
...,...,...
107,misschien nog,4
108,om dit,4
109,alles over,3
110,echt heel,3


In [30]:
df_threegram_for_link = df_threegram_select[df_threegram_select["threegram"].isin(threegram_list)]
df_threegram_for_link.reset_index(drop=True, inplace=True)
df_threegram_for_link

,threegram,frequency
0,dat zal wel,4067
1,niet voor mij,2035
2,ja ja ja,1897
3,wat was het,1812
4,we doen het,1758
5,ik weet dat,1019
6,dit is echt,971
7,dan gaan we,916
8,maar ik dacht,508
9,was het goed,226


In [31]:
df_threegram_selected_for_link = word_usage_result(word_list, df_threegram_for_link, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_selected_for_link 

,threegram,frequency
0,dat zal wel,4067
1,niet voor mij,2035
2,ja ja ja,1897
3,wat was het,1812
4,we doen het,1758
5,ik weet dat,1019
6,dit is echt,971
7,dan gaan we,916
8,maar ik dacht,508
9,was het goed,226


In [32]:
df_twogram_search_result = word_group_youtube(df_youtube_sentence, df_twogram_selected_for_link["twogram"], "sentence", sample_num)
df_twogram_search_result

,search_string,start_time,end_time,sentence,video_id
0,dank u,5009.770,5013.020,hamster wethouder ja dank u wel voorzitter dat...,uJhEbO42qRo
1,nog niet,104.950,111.543,maar kan jij even uitleggen voor de kijkers th...,5S5QX4ZK3Q0
2,ja ja,2886.510,2895.300,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4
3,niet nu,12560.160,12562.520,dat is niet nu pas begonnen daar zijn we al we...,QAjt9liE_a4
4,doe maar,3919.190,3922.910,oké doe maar wat je zegt is dat ontwikkeling g...,EmxRrMXp3UY
...,...,...,...,...,...
107,misschien nog,1771.039,1777.399,hebben bij provincies en gemeenten zal dat mis...,KFkJCia8L5M
108,om dit,242.435,248.761,om dit niet ongemerkt voorbij te laten gaan he...,OIb3c0hWNrI
109,alles over,744.855,747.515,je er alles over maar als je zou willen,wJZCHf6RQ8c
110,echt heel,625.859,628.899,dan echt heel strak elke dag gewoon,yqD5ixgnKqI


In [33]:
df_word_group_time_loc_twogram_result = word_group_time_loc(df_twogram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,dank u,5010.543810,5010.853333,hamster wethouder ja dank u wel voorzitter dat...,uJhEbO42qRo
1,nog niet,107.980653,108.512347,maar kan jij even uitleggen voor de kijkers th...,5S5QX4ZK3Q0
2,ja ja,2889.629032,2890.290645,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4
3,niet nu,12560.400000,12560.760000,dat is niet nu pas begonnen daar zijn we al we...,QAjt9liE_a4
4,doe maar,3919.349429,3919.880857,oké doe maar wat je zegt is dat ontwikkeling g...,EmxRrMXp3UY
...,...,...,...,...,...
107,misschien nog,1774.648730,1775.937919,hebben bij provincies en gemeenten zal dat mis...,KFkJCia8L5M
108,om dit,242.435000,242.955965,om dit niet ongemerkt voorbij te laten gaan he...,OIb3c0hWNrI
109,alles over,745.196026,746.014487,je er alles over maar als je zou willen,wJZCHf6RQ8c
110,echt heel,626.119571,627.075000,dan echt heel strak elke dag gewoon,yqD5ixgnKqI


In [34]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,dank u,5009.943810,5011.453333,hamster wethouder ja dank u wel voorzitter dat...,uJhEbO42qRo
1,nog niet,107.380653,109.112347,maar kan jij even uitleggen voor de kijkers th...,5S5QX4ZK3Q0
2,ja ja,2889.029032,2890.890645,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4
3,niet nu,12559.800000,12561.360000,dat is niet nu pas begonnen daar zijn we al we...,QAjt9liE_a4
4,doe maar,3918.749429,3920.480857,oké doe maar wat je zegt is dat ontwikkeling g...,EmxRrMXp3UY
...,...,...,...,...,...
107,misschien nog,1774.048730,1776.537919,hebben bij provincies en gemeenten zal dat mis...,KFkJCia8L5M
108,om dit,241.835000,243.555965,om dit niet ongemerkt voorbij te laten gaan he...,OIb3c0hWNrI
109,alles over,744.596026,746.614487,je er alles over maar als je zou willen,wJZCHf6RQ8c
110,echt heel,625.519571,627.675000,dan echt heel strak elke dag gewoon,yqD5ixgnKqI


In [35]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result 

,search_string,start_time,end_time,sentence,video_id
0,dank u,5010,5011,hamster wethouder ja dank u wel voorzitter dat...,uJhEbO42qRo
1,nog niet,107,109,maar kan jij even uitleggen voor de kijkers th...,5S5QX4ZK3Q0
2,ja ja,2889,2891,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4
3,niet nu,12560,12561,dat is niet nu pas begonnen daar zijn we al we...,QAjt9liE_a4
4,doe maar,3919,3920,oké doe maar wat je zegt is dat ontwikkeling g...,EmxRrMXp3UY
...,...,...,...,...,...
107,misschien nog,1774,1777,hebben bij provincies en gemeenten zal dat mis...,KFkJCia8L5M
108,om dit,242,244,om dit niet ongemerkt voorbij te laten gaan he...,OIb3c0hWNrI
109,alles over,745,747,je er alles over maar als je zou willen,wJZCHf6RQ8c
110,echt heel,626,628,dan echt heel strak elke dag gewoon,yqD5ixgnKqI


In [36]:
df_word_group_time_loc_twogram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_twogram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_twogram_result['start_time'].map(str)+"s"
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,dank u,5010,5011,hamster wethouder ja dank u wel voorzitter dat...,uJhEbO42qRo,https://www.youtube.com/watch?v=uJhEbO42qRo&t=...
1,nog niet,107,109,maar kan jij even uitleggen voor de kijkers th...,5S5QX4ZK3Q0,https://www.youtube.com/watch?v=5S5QX4ZK3Q0&t=...
2,ja ja,2889,2891,zo lekker om dat te kunnen zeggen ja ja nee ik...,wehip3eUEi4,https://www.youtube.com/watch?v=wehip3eUEi4&t=...
3,niet nu,12560,12561,dat is niet nu pas begonnen daar zijn we al we...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
4,doe maar,3919,3920,oké doe maar wat je zegt is dat ontwikkeling g...,EmxRrMXp3UY,https://www.youtube.com/watch?v=EmxRrMXp3UY&t=...
...,...,...,...,...,...,...
107,misschien nog,1774,1777,hebben bij provincies en gemeenten zal dat mis...,KFkJCia8L5M,https://www.youtube.com/watch?v=KFkJCia8L5M&t=...
108,om dit,242,244,om dit niet ongemerkt voorbij te laten gaan he...,OIb3c0hWNrI,https://www.youtube.com/watch?v=OIb3c0hWNrI&t=...
109,alles over,745,747,je er alles over maar als je zou willen,wJZCHf6RQ8c,https://www.youtube.com/watch?v=wJZCHf6RQ8c&t=...
110,echt heel,626,628,dan echt heel strak elke dag gewoon,yqD5ixgnKqI,https://www.youtube.com/watch?v=yqD5ixgnKqI&t=...


In [37]:
df_word_group_time_loc_twogram_result.to_excel(f"{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [38]:
df_threegram_search_result = word_group_youtube(df_youtube_sentence, df_threegram_selected_for_link["threegram"], "sentence", sample_num)
df_threegram_search_result

,search_string,start_time,end_time,sentence,video_id
0,dat zal wel,369.600,372.446,maar dat zal wel goed komen ik vind jou nu in ...,HKVUi2B3AV4
1,niet voor mij,4365.660,4370.160,is waren ze ontkende te nemen jullie hoeven ni...,Co4vE_sSnZM
2,ja ja ja,617.880,621.030,dat is bijna onmogelijk ja ja ja precies,ULhra7fWWqE
3,wat was het,3261.540,3270.330,natuurlijk langs verschillende kanten bekijken...,sMaUZFkQ6QM
4,we doen het,1634.750,1638.760,het antwoord was elke keer dat dat niet nodig ...,QAjt9liE_a4
5,ik weet dat,372.390,378.210,weg kon dan heb ik geen genoeg dus om nu ze we...,ftwsaGlOEdA
6,dit is echt,412.131,418.560,dit is echt een rat kijk maar naar het hoofd,IIpCTf5UAMA
7,dan gaan we,771.680,772.940,en dan gaan we een bordje maken,IVUi52Ejpfs
8,maar ik dacht,41.500,48.180,maar ik dacht als ik de hoek van mijn bladen,fphTOzDV2ZQ
9,was het goed,223.400,225.260,als de deur niet klemde was het goed ok,hx_YcOUrtPk


In [39]:
df_word_group_time_loc_threegram_result = word_group_time_loc(df_threegram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,dat zal wel,369.764986,370.301188,maar dat zal wel goed komen ik vind jou nu in ...,HKVUi2B3AV4
1,niet voor mij,4367.614545,4368.296364,is waren ze ontkende te nemen jullie hoeven ni...,Co4vE_sSnZM
2,ja ja ja,619.691250,620.478750,dat is bijna onmogelijk ja ja ja precies,ULhra7fWWqE
3,wat was het,3265.624242,3266.778485,natuurlijk langs verschillende kanten bekijken...,sMaUZFkQ6QM
4,we doen het,1638.034787,1638.589362,het antwoord was elke keer dat dat niet nodig ...,QAjt9liE_a4
5,ik weet dat,377.104937,378.062658,weg kon dan heb ik geen genoeg dus om nu ze we...,ftwsaGlOEdA
6,dit is echt,412.131000,413.884364,dit is echt een rat kijk maar naar het hoofd,IIpCTf5UAMA
7,dan gaan we,771.761290,772.289677,en dan gaan we een bordje maken,IVUi52Ejpfs
8,maar ik dacht,41.500000,43.625455,maar ik dacht als ik de hoek van mijn bladen,fphTOzDV2ZQ
9,was het goed,224.496923,225.164615,als de deur niet klemde was het goed ok,hx_YcOUrtPk


In [40]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,dat zal wel,369.164986,370.901188,maar dat zal wel goed komen ik vind jou nu in ...,HKVUi2B3AV4
1,niet voor mij,4367.014545,4368.896364,is waren ze ontkende te nemen jullie hoeven ni...,Co4vE_sSnZM
2,ja ja ja,619.091250,621.078750,dat is bijna onmogelijk ja ja ja precies,ULhra7fWWqE
3,wat was het,3265.024242,3267.378485,natuurlijk langs verschillende kanten bekijken...,sMaUZFkQ6QM
4,we doen het,1637.434787,1639.189362,het antwoord was elke keer dat dat niet nodig ...,QAjt9liE_a4
5,ik weet dat,376.504937,378.662658,weg kon dan heb ik geen genoeg dus om nu ze we...,ftwsaGlOEdA
6,dit is echt,411.531000,414.484364,dit is echt een rat kijk maar naar het hoofd,IIpCTf5UAMA
7,dan gaan we,771.161290,772.889677,en dan gaan we een bordje maken,IVUi52Ejpfs
8,maar ik dacht,40.900000,44.225455,maar ik dacht als ik de hoek van mijn bladen,fphTOzDV2ZQ
9,was het goed,223.896923,225.764615,als de deur niet klemde was het goed ok,hx_YcOUrtPk


In [41]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result 

,search_string,start_time,end_time,sentence,video_id
0,dat zal wel,369,371,maar dat zal wel goed komen ik vind jou nu in ...,HKVUi2B3AV4
1,niet voor mij,4367,4369,is waren ze ontkende te nemen jullie hoeven ni...,Co4vE_sSnZM
2,ja ja ja,619,621,dat is bijna onmogelijk ja ja ja precies,ULhra7fWWqE
3,wat was het,3265,3267,natuurlijk langs verschillende kanten bekijken...,sMaUZFkQ6QM
4,we doen het,1637,1639,het antwoord was elke keer dat dat niet nodig ...,QAjt9liE_a4
5,ik weet dat,377,379,weg kon dan heb ik geen genoeg dus om nu ze we...,ftwsaGlOEdA
6,dit is echt,412,414,dit is echt een rat kijk maar naar het hoofd,IIpCTf5UAMA
7,dan gaan we,771,773,en dan gaan we een bordje maken,IVUi52Ejpfs
8,maar ik dacht,41,44,maar ik dacht als ik de hoek van mijn bladen,fphTOzDV2ZQ
9,was het goed,224,226,als de deur niet klemde was het goed ok,hx_YcOUrtPk


In [42]:
df_word_group_time_loc_threegram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_threegram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_threegram_result['start_time'].map(str)+"s"
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,dat zal wel,369,371,maar dat zal wel goed komen ik vind jou nu in ...,HKVUi2B3AV4,https://www.youtube.com/watch?v=HKVUi2B3AV4&t=...
1,niet voor mij,4367,4369,is waren ze ontkende te nemen jullie hoeven ni...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...
2,ja ja ja,619,621,dat is bijna onmogelijk ja ja ja precies,ULhra7fWWqE,https://www.youtube.com/watch?v=ULhra7fWWqE&t=...
3,wat was het,3265,3267,natuurlijk langs verschillende kanten bekijken...,sMaUZFkQ6QM,https://www.youtube.com/watch?v=sMaUZFkQ6QM&t=...
4,we doen het,1637,1639,het antwoord was elke keer dat dat niet nodig ...,QAjt9liE_a4,https://www.youtube.com/watch?v=QAjt9liE_a4&t=...
5,ik weet dat,377,379,weg kon dan heb ik geen genoeg dus om nu ze we...,ftwsaGlOEdA,https://www.youtube.com/watch?v=ftwsaGlOEdA&t=...
6,dit is echt,412,414,dit is echt een rat kijk maar naar het hoofd,IIpCTf5UAMA,https://www.youtube.com/watch?v=IIpCTf5UAMA&t=...
7,dan gaan we,771,773,en dan gaan we een bordje maken,IVUi52Ejpfs,https://www.youtube.com/watch?v=IVUi52Ejpfs&t=...
8,maar ik dacht,41,44,maar ik dacht als ik de hoek van mijn bladen,fphTOzDV2ZQ,https://www.youtube.com/watch?v=fphTOzDV2ZQ&t=41s
9,was het goed,224,226,als de deur niet klemde was het goed ok,hx_YcOUrtPk,https://www.youtube.com/watch?v=hx_YcOUrtPk&t=...


In [43]:
df_word_group_time_loc_threegram_result.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [44]:
df_word_search_result = word_group_youtube(df_youtube_sentence, word_list, "sentence", sample_num)
df_word_search_result

,search_string,start_time,end_time,sentence,video_id
0,ik,196.073,199.020,waar ik ook zelf veel persoonlijke voorbeelden...,pruDyt__jYs
1,je,93.979,97.850,nou dat is wel heel specifiek voor het mbo dat...,smo9IsqShlc
2,het,1109.030,1116.440,zwart je ziet het hier overlopen in elkaar en ...,CqaNlQx_CmE
3,de,990.280,996.760,wat mistige beeld je fotos zal minder scherp o...,5XGbMbvvfyY
4,dat,128.067,133.910,en ja ook als burgemeester hoop ik dat deze lo...,e4K8PJpTssk
...,...,...,...,...,...
195,genoeg,1303.870,1308.790,in de praktijk gewoon niet zo blijkt te zijn e...,GOkG8Yq9ANo
196,wist,1644.399,1647.878,wist dat meisje zich vier getallen te onthoude...,XRBb17aWENc
197,oh,1921.920,1925.840,dan denk ik oh ja,xeAlsYp5XX0
198,klaar,10440.726,10443.170,en heren daarmee zijn wij klaar met de,R4Q1cNbNKU0


In [45]:
df_word_group_time_loc_word_result = word_group_time_loc(df_word_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,ik,196.287327,196.501655,waar ik ook zelf veel persoonlijke voorbeelden...,pruDyt__jYs
1,je,96.124373,96.310928,nou dat is wel heel specifiek voor het mbo dat...,smo9IsqShlc
2,het,1110.100333,1110.512000,zwart je ziet het hier overlopen in elkaar en ...,CqaNlQx_CmE
3,de,994.674483,994.972414,wat mistige beeld je fotos zal minder scherp o...,5XGbMbvvfyY
4,dat,130.350471,130.686276,en ja ook als burgemeester hoop ik dat deze lo...,e4K8PJpTssk
...,...,...,...,...,...
195,genoeg,1307.806000,1308.243333,in de praktijk gewoon niet zo blijkt te zijn e...,GOkG8Yq9ANo
196,wist,1644.399000,1644.624909,wist dat meisje zich vier getallen te onthoude...,XRBb17aWENc
197,oh,1924.456471,1925.378824,dan denk ik oh ja,xeAlsYp5XX0
198,klaar,10442.333895,10442.784105,en heren daarmee zijn wij klaar met de,R4Q1cNbNKU0


In [46]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,ik,195.687327,197.101655,waar ik ook zelf veel persoonlijke voorbeelden...,pruDyt__jYs
1,je,95.524373,96.910928,nou dat is wel heel specifiek voor het mbo dat...,smo9IsqShlc
2,het,1109.500333,1111.112000,zwart je ziet het hier overlopen in elkaar en ...,CqaNlQx_CmE
3,de,994.074483,995.572414,wat mistige beeld je fotos zal minder scherp o...,5XGbMbvvfyY
4,dat,129.750471,131.286276,en ja ook als burgemeester hoop ik dat deze lo...,e4K8PJpTssk
...,...,...,...,...,...
195,genoeg,1307.206000,1308.843333,in de praktijk gewoon niet zo blijkt te zijn e...,GOkG8Yq9ANo
196,wist,1643.799000,1645.224909,wist dat meisje zich vier getallen te onthoude...,XRBb17aWENc
197,oh,1923.856471,1925.978824,dan denk ik oh ja,xeAlsYp5XX0
198,klaar,10441.733895,10443.384105,en heren daarmee zijn wij klaar met de,R4Q1cNbNKU0


In [47]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result 

,search_string,start_time,end_time,sentence,video_id
0,ik,196,197,waar ik ook zelf veel persoonlijke voorbeelden...,pruDyt__jYs
1,je,96,97,nou dat is wel heel specifiek voor het mbo dat...,smo9IsqShlc
2,het,1110,1111,zwart je ziet het hier overlopen in elkaar en ...,CqaNlQx_CmE
3,de,994,996,wat mistige beeld je fotos zal minder scherp o...,5XGbMbvvfyY
4,dat,130,131,en ja ook als burgemeester hoop ik dat deze lo...,e4K8PJpTssk
...,...,...,...,...,...
195,genoeg,1307,1309,in de praktijk gewoon niet zo blijkt te zijn e...,GOkG8Yq9ANo
196,wist,1644,1645,wist dat meisje zich vier getallen te onthoude...,XRBb17aWENc
197,oh,1924,1926,dan denk ik oh ja,xeAlsYp5XX0
198,klaar,10442,10443,en heren daarmee zijn wij klaar met de,R4Q1cNbNKU0


In [48]:
df_word_group_time_loc_word_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_word_result['video_id'].map(str)+"&t="+df_word_group_time_loc_word_result['start_time'].map(str)+"s"
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik,196,197,waar ik ook zelf veel persoonlijke voorbeelden...,pruDyt__jYs,https://www.youtube.com/watch?v=pruDyt__jYs&t=...
1,je,96,97,nou dat is wel heel specifiek voor het mbo dat...,smo9IsqShlc,https://www.youtube.com/watch?v=smo9IsqShlc&t=96s
2,het,1110,1111,zwart je ziet het hier overlopen in elkaar en ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
3,de,994,996,wat mistige beeld je fotos zal minder scherp o...,5XGbMbvvfyY,https://www.youtube.com/watch?v=5XGbMbvvfyY&t=...
4,dat,130,131,en ja ook als burgemeester hoop ik dat deze lo...,e4K8PJpTssk,https://www.youtube.com/watch?v=e4K8PJpTssk&t=...
...,...,...,...,...,...,...
195,genoeg,1307,1309,in de praktijk gewoon niet zo blijkt te zijn e...,GOkG8Yq9ANo,https://www.youtube.com/watch?v=GOkG8Yq9ANo&t=...
196,wist,1644,1645,wist dat meisje zich vier getallen te onthoude...,XRBb17aWENc,https://www.youtube.com/watch?v=XRBb17aWENc&t=...
197,oh,1924,1926,dan denk ik oh ja,xeAlsYp5XX0,https://www.youtube.com/watch?v=xeAlsYp5XX0&t=...
198,klaar,10442,10443,en heren daarmee zijn wij klaar met de,R4Q1cNbNKU0,https://www.youtube.com/watch?v=R4Q1cNbNKU0&t=...


In [49]:
df_word_group_time_loc_word_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False)

#### Copy Move And Delete

In [50]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx")
output_file

['Dutch_Twogram_With_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx',
 'Dutch_Threegram_With_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx',
 'Dutch_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx']

In [51]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [52]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass